# 这是一个用来模拟储能充放策略算法的模拟程序

![规则集说明图](./规则集-第一版.png)

In [1]:
from test_ruleset_battery import *
from pandas.core.frame import DataFrame
import pandas as pd

In [2]:
'''
模拟程序输入：df(p_pvlist, p_loadlist, c_outlist), c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max
分别对应的是：光伏a、负荷序列a，分时电价序列a，光伏上网电价，初始的SOC，SOC上下限，电池充上限功率，电池放上限功率

模拟程序输出：(p_batlist, p_上网list, p_下网list), modelist, soclist, targetlist
分别对应的是：策略a（电池充放功率，上网功率，下网功率），工作模式a，储能电池容量a, 经济效益指标a

---> 每一步策略生成之间包含一个soc的状态转移方程（类似）
 
带a的表示是时间序列格式的数据
不带a的表示不是时间序列，就是一个值

'''
def moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max):
    
    times=df.shape[0]#获取行数，即知道需要循环执行几次调度算法，for i in range(1,times):执行一次调度算法，times+1
    
    #循环执行times次的调度算法，每次执行完毕之后做一下soc的计算，用于下一次循环的输入
    '''
    执行一次储能充放策略调度算法（可以是基于规则集的也可以是基于线性规划的）：
    schedule_bat_rule((p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max),return 
    return p_bat,p_上网,p_下网,mode,mode_dict[mode])
    '''     
    p_batlist=[]
    p_上网list=[]
    p_下网list=[]
    modelist=[]
    targetlist=[]
    soclist=[]
        
    for i in range(1, times+1):
        p_pv=df.iloc[i-1,0]
        p_load=df.iloc[i-1,1]
        c_out=df.iloc[i-1,2]
        
        p_bat_now=schedule_bat_rule(p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max)[0]
        p_上网_now=schedule_bat_rule(p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max)[1]
        p_下网_now=schedule_bat_rule(p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max)[2]
        mode_now=schedule_bat_rule(p_pv,p_load,c_out,c_pv,soc,soc_max,soc_min,p_bat_in_max,p_bat_out_max)[3]
        
        p_batlist.append(p_bat_now)
        p_上网list.append(p_上网_now)
        p_下网list.append(p_下网_now)
        modelist.append(mode_now)

        soc=soc-p_bat_now*0.5
        
        soclist.append(soc)         
        
    return p_batlist, p_上网list, p_下网list, modelist, soclist, targetlist

# 准备测试所需的输入数据

In [22]:
df=pd.read_excel("C:/Users/79918/Desktop/新建文件夹/new_ems/doc/算法相关/【模拟数据算法小测-xt】.xlsx",sheet_name="模拟数据")
df=df.iloc[:,[0,1,2]]

In [23]:
df.head()

,p_pv,p_load,c_out
0,0,27.563387,0.266
1,0,25.563076,0.266
2,0,26.165248,0.266
3,0,29.008954,0.266
4,0,26.973848,0.266


In [24]:
df=df
c_pv=0.415
soc=30
soc_max=95
soc_min=5
p_bat_in_max=20
p_bat_out_max=20

# 测试

In [25]:
#获取模拟程序的结果
p_batlist=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[0]
p_上网list=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[1]
p_下网list=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[2]
modelist=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[3]
soclist=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[4]
#targetlist=moc_program(df, c_pv, soc, soc_max, soc_min, p_bat_in_max, p_bat_out_max)[5]

In [26]:
#对结果进行整理
#把工作模式对应的简单文字说明加到results里面

modetextlist=[]
for i in modelist:
    modetextlist.append(mode_dict[i])

results={
    "电池充放功率指令" : p_batlist,
    "上网功率" : p_上网list,
    "下网功率" : p_下网list,
    "工作模式" : modelist,
    "工作模式说明" : modetextlist,
    "下一步soc%" : soclist
#    "经济效益指标" : targetlist
}

results=DataFrame(results)#将字典转换成为数据框

#把整理完毕的结果输出看一下
results

,电池充放功率指令,上网功率,下网功率,工作模式,工作模式说明,下一步soc%
0,-20.0,0.0,47.563387,5,负荷>=光伏，外网电充进电池，负荷不足电网供电,40.00000
1,-20.0,0.0,45.563076,5,负荷>=光伏，外网电充进电池，负荷不足电网供电,50.00000
2,-20.0,0.0,46.165248,5,负荷>=光伏，外网电充进电池，负荷不足电网供电,60.00000
3,-20.0,0.0,49.008954,5,负荷>=光伏，外网电充进电池，负荷不足电网供电,70.00000
4,-20.0,0.0,46.973848,5,负荷>=光伏，外网电充进电池，负荷不足电网供电,80.00000
...,...,...,...,...,...,...
163,20.0,0.0,11.499202,8,负荷>=光伏，电池放电给负荷，剩余负荷不足由电网供电,59.21434
164,20.0,0.0,10.960550,8,负荷>=光伏，电池放电给负荷，剩余负荷不足由电网供电,49.21434
165,20.0,0.0,9.981028,8,负荷>=光伏，电池放电给负荷，剩余负荷不足由电网供电,39.21434
166,-20.0,0.0,47.507491,5,负荷>=光伏，外网电充进电池，负荷不足电网供电,49.21434


In [27]:
results.to_csv("results-1101-1107.csv")